## load data

In [1]:
import sys
sys.path.append('../../')

In [2]:
import json
import pandas as pd
from mlinterpreter.util import preprocess_data

In [3]:
# 读入数据 和 schema
data_path = "../../demo_data/adult.data"
schema_path = "../../demo_data/schema.json"
df = pd.read_csv(data_path, header=None)
df.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex",
                 "capital-gain","capital-loss","hours-per-week","native-country","income"]
with open(schema_path, "r") as f:
    schema = json.load(f)

## preprocess data

In [4]:
# 预处理数据
preprocess_df, preprocess_dict = preprocess_data(df=df, schema=schema, process_label=True)

## influential instances

获取最有影响样本
    
算法说明：
   - 将数据集切分成 train 和 valid
   - 在train上训练NN模型
   - 计算将train中某个样本剔除后，valid上loss的变化大小
       - 这里都是用近似估计的方式计算valid上loss的变化
 
算法(get_influential_instances_nn):
[Understanding black-box predictions via influence functions,(ICML'17)]
   - 通过Upweighting a training point和Stochastic estimation计算train样本对valid loss的影响



In [5]:
from mlinterpreter.instance.influence import get_influential_instances_nn

In [6]:
top10_influential_instances, influence  = get_influential_instances_nn(
    preprocess_df[:500], 
    df_display=df[:500], 
    schema=schema, 
    split=0.2, 
    topk=10)

Epoch 1/10
2/2 [==============================] - 4s 815ms/step - loss: 10.2580 - auc: 0.4973 - val_loss: 0.4912 - val_auc: 0.5832
Epoch 2/10
2/2 [==============================] - 0s 43ms/step - loss: 0.6012 - auc: 0.5788 - val_loss: 0.5017 - val_auc: 0.7709
Epoch 3/10
2/2 [==============================] - 0s 43ms/step - loss: 0.5173 - auc: 0.7487 - val_loss: 0.3581 - val_auc: 0.8538
Epoch 4/10
2/2 [==============================] - 0s 42ms/step - loss: 0.4024 - auc: 0.8359 - val_loss: 0.3542 - val_auc: 0.8728
Epoch 5/10
2/2 [==============================] - 0s 41ms/step - loss: 0.3556 - auc: 0.8948 - val_loss: 0.3627 - val_auc: 0.8529
Epoch 00005: early stopping
Runing calculate inverse hvp with lissa at iter 1/1


100%|██████████| 5000/5000 [10:23<00:00,  8.01it/s]


In [7]:
print(top10_influential_instances)

     age     workclass  fnlwgt      education  education-num  \
426   36   Federal-gov  255191        Masters             14   
200   26       Private  397317        Masters             14   
222   90       Private   51744        HS-grad              9   
23    43       Private  117037           11th              7   
170   27       Private  124953        HS-grad              9   
32    45       Private  386940      Bachelors             13   
183   40       Private  286370        7th-8th              4   
215   37       Private  635913      Bachelors             13   
387   44       Private  162028   Some-college             10   
143   45       Private  196584      Assoc-voc             11   

          marital-status          occupation    relationship    race      sex  \
426        Never-married      Prof-specialty   Not-in-family   White     Male   
200        Never-married      Prof-specialty   Not-in-family   White   Female   
222        Never-married       Other-service   Not-i